<a href="https://colab.research.google.com/github/chakrateja70/Agent/blob/main/ClaudeAgent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
https://chatgpt.com/c/67a198f6-fd28-800c-9a08-0e0dd41e3ac4

# *Required Pacakges*

In [ ]:
!pip install langchain pinecone-client anthropic voyageai langchain-voyageai python-dotenv pypdf langchain-community


In [12]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader, TextLoader

# Load documents (PDF, TXT, etc.)
pdf_loader = PyPDFLoader("/content/Visual Search for Jewellery.pdf")
#txt_loader = TextLoader("sample.txt")

# Combine all documents
documents = pdf_loader.load()

# Split into smaller chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
splits = text_splitter.split_documents(documents)

print(f"Total Chunks Created: {len(splits)}")


Total Chunks Created: 28
